# Kpay Case Study - Merchant Data Analysis
## Business Development Strategy for Australian Merchants

## 1. Load & Inspect Data

In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#load dataset
FILE_PATH = "data/Case Study - Case Study.csv"
df = pd.read_csv(FILE_PATH, dtype=str)  #preserves phone number formatting

print(f"Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")

Loaded: 199,999 rows × 10 columns

Columns: ['id', 'lead_key', 'phone', 'business_name', 'state', 'suburb', 'address', 'sector_level_1', 'sector_level_2', 'sector_level_3']


In [2]:
df.head()

,id,lead_key,phone,business_name,state,suburb,address,sector_level_1,sector_level_2,sector_level_3
0,2312630,L_611287243333,611287000000,PRO IT,NSW,sydney,"Suite 604, Level 6/83 York Street Sydney NSW 2000",Retail,Electronics & Appliances,"Computer, IT Technical Support"
1,2312631,L_611300048153,611300000000,Hoist Care,NSW,sydney,Sydney Sydney NSW 2000,Retail,Fashion & Accessories,"Wholesale Car Accessories, Manufacturers"
2,2312632,L_611300053384,611300000000,JEEVI,NSW,sydney,714/368 Sussex Street Sydney NSW 2000,Beauty & Wellness,Others,Community Health Services
3,2312633,L_611300069313,611300000000,Dunlap Bike Finance,NSW,sydney,377 Kent St Sydney NSW 2000,Retail,Fashion & Accessories,"Wholesale Motorcycle Parts and Accessories, Ma..."
4,2312634,L_611300074353,611300000000,Metiri Mensus,NSW,parramatta,Suite 107 L 1 30 Cowper Street Parramatta NSW...,F&B,Others,Food


In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   id              199999 non-null  str  
 1   lead_key        199999 non-null  str  
 2   phone           199999 non-null  str  
 3   business_name   199989 non-null  str  
 4   state           191664 non-null  str  
 5   suburb          94988 non-null   str  
 6   address         199945 non-null  str  
 7   sector_level_1  162229 non-null  str  
 8   sector_level_2  162229 non-null  str  
 9   sector_level_3  193153 non-null  str  
dtypes: str(10)
memory usage: 15.3 MB


In [4]:
df.describe(include='all')

,id,lead_key,phone,business_name,state,suburb,address,sector_level_1,sector_level_2,sector_level_3
count,199999,199999,199999,199989,191664,94988,199945,162229,162229,193153
unique,199999,199999,199665,189060,70,4393,175908,146,1554,7191
top,2312630,L_611287243333,611301000000,The Lott,NSW,Sydney,"Sydney,NSW,2000",Others,Others,Investing
freq,1,1,157,126,113341,3128,281,82508,91005,9918


In [5]:
print(f"\n=== DATA TYPES ===")
print(df.dtypes)


=== DATA TYPES ===
id                str
lead_key          str
phone             str
business_name     str
state             str
suburb            str
address           str
sector_level_1    str
sector_level_2    str
sector_level_3    str
dtype: object


## 2. Data Quality Check

In [6]:
print("=" * 55)
print("  DATA QUALITY CHECK — PART 1A")
print("=" * 55)

total = len(df)

# 1. Missing values
print("\n📊 MISSING VALUES:")
missing = df.isnull().sum()
missing_pct = (missing / total * 100).round(1)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])

# 2. Duplicates
dup_lead_key = df.duplicated(subset=['lead_key']).sum()
dup_name_suburb = df.duplicated(subset=['business_name', 'suburb']).sum()
print(f"\n📊 DUPLICATES:")
print(f"  lead_key duplicates:              {dup_lead_key:,}")
print(f"  business_name + suburb dupes:     {dup_name_suburb:,}")

# 3. Phone analysis — THE KEY FINDING
print(f"\n📊 PHONE ANALYSIS:")
print(f"  Total records:                    {total:,}")
print(f"\n  Top 20 most common phone values:")
print(df['phone'].value_counts().head(20).to_string())

# Masked phone patterns
masked_patterns = ['611300000000', '611287000000', '611800000000']
df['phone_masked'] = df['phone'].str.strip().isin(masked_patterns) | \
                     df['phone'].str.match(r'^6113000\d{5}$', na=False) | \
                     df['phone'].isna()

masked_count = df['phone_masked'].sum()
masked_pct = masked_count / total * 100
print(f"\n  ⚠️  Masked/invalid phones:          {masked_count:,} ({masked_pct:.1f}%)")
print(f"  ✅ Potentially valid phones:        {total - masked_count:,} ({100 - masked_pct:.1f}%)")

# 4. Sector breakdown
print(f"\n📊 SECTOR BREAKDOWN (sector_level_1):")
print(df['sector_level_1'].value_counts().to_string())

# 5. Geographic spread
print(f"\n📊 TOP 20 SUBURBS:")
print(df['suburb'].str.lower().value_counts().head(20).to_string())

# 6. State breakdown
print(f"\n📊 STATE BREAKDOWN:")
print(df['state'].value_counts().to_string())

  DATA QUALITY CHECK — PART 1A

📊 MISSING VALUES:
                Missing Count  Missing %
business_name              10        0.0
state                    8335        4.2
suburb                 105011       52.5
address                    54        0.0
sector_level_1          37770       18.9
sector_level_2          37770       18.9
sector_level_3           6846        3.4

📊 DUPLICATES:
  lead_key duplicates:              0
  business_name + suburb dupes:     6,923

📊 PHONE ANALYSIS:
  Total records:                    199,999

  Top 20 most common phone values:
phone
611301000000       157
611300000000       105
611801000000        41
611800000000        31
610292000000         2
613832000000         2
613833000000         2
616386000000         2
611287000000         1
+61 408 268 946      1
+61 2 4939 3300      1
+61 459 484 837      1
(02) 5504 5491       1
0456 378 130         1
(02) 6581 0544       1
0400 745 366         1
+61 411 665 644      1
+61 414 500 589      1
0499 521


  ⚠️  Masked/invalid phones:          137 (0.1%)
  ✅ Potentially valid phones:        199,862 (99.9%)

📊 SECTOR BREAKDOWN (sector_level_1):
sector_level_1
Others                            82508
Retail                            26333
F&B                               18306
Professional Services             16749
Beauty & Wellness                  9444
Retail Shopping                     897
Food & Drink                        782
Restaurants                         778
Hair & Beauty                       722
Restaurant                          717
Furniture                           627
Restaurants & Bistros               505
Clothing                            328
Food & Beverage                     248
Beauty & Personal Care              213
Food Retailers                      170
Fashion                             162
Stores                              145
Electronics                         144
Printers                            142
Cafes                               121
Heal

suburb
sydney            6840
parramatta        1546
surry hills       1342
chatswood         1206
bankstown         1070
marrickville       994
castle hill        988
liverpool          980
blacktown          961
brookvale          888
auburn             840
alexandria         838
north sydney       807
burwood            779
hornsby            710
newtown            709
bondi junction     678
haymarket          659
darlinghurst       651
hurstville         647

📊 STATE BREAKDOWN:
state
NSW                             113341
VIC                              33292
QLD                              21751
WA                                9660
SA                                7833
ACT                               2312
TAS                               1708
NT                                 588
New South Wales                    523
KY                                 111
NY                                  77
CA                                  49
FL                                  44


## 3. Business Impact Summary

In [7]:
print("=" * 55)
print("  BUSINESS IMPACT — BD COST OF DATA ISSUES")
print("=" * 55)

total = len(df)
masked_count = df['phone_masked'].sum()
dup_count = df.duplicated(subset=['lead_key']).sum()

# Non-target sectors
non_target_keywords = ['Retirement', 'Equestrian', 'NGO', 'Construction', 
                       'Community Health', 'Religious', 'Fabrication',
                       'Marriage Celebrant', 'Recycling']
non_target_mask = df['sector_level_3'].str.contains(
    '|'.join(non_target_keywords), case=False, na=False)
non_target_count = non_target_mask.sum()

# Others in sector_level_1
others_count = (df['sector_level_1'] == 'Others').sum()

print(f"""
┌─────────────────────────────────────────────────────────┐
│  ISSUE              │ COUNT      │ BD IMPACT             │
├─────────────────────────────────────────────────────────┤
│  Masked phones      │ {masked_count:>8,}   │ Uncontactable leads   │
│  Duplicate records  │ {dup_count:>8,}   │ Wasted repeat outreach│  
│  Non-target sectors │ {non_target_count:>8,}   │ Wrong pitch, low ROI  │
│  'Others' sector    │ {others_count:>8,}   │ Can't prioritise      │
└─────────────────────────────────────────────────────────┘

💡 KEY FINDING FOR SLIDE 3:
   Without cleaning: {masked_count + dup_count + non_target_count:,} records 
   ({((masked_count + dup_count + non_target_count)/total*100):.0f}%) would waste BD effort.
   
   Actionable Lead Rate BEFORE cleaning: 
   {((total - masked_count - non_target_count)/total*100):.1f}%
""")

  BUSINESS IMPACT — BD COST OF DATA ISSUES



┌─────────────────────────────────────────────────────────┐
│  ISSUE              │ COUNT      │ BD IMPACT             │
├─────────────────────────────────────────────────────────┤
│  Masked phones      │      137   │ Uncontactable leads   │
│  Duplicate records  │        0   │ Wasted repeat outreach│  
│  Non-target sectors │    2,394   │ Wrong pitch, low ROI  │
│  'Others' sector    │   82,508   │ Can't prioritise      │
└─────────────────────────────────────────────────────────┘

💡 KEY FINDING FOR SLIDE 3:
   Without cleaning: 2,531 records 
   (1%) would waste BD effort.

   Actionable Lead Rate BEFORE cleaning: 
   98.7%



## 4. Data Cleaning

In [8]:
import re
import phonenumbers
from phonenumbers import PhoneNumberType

print("=" * 55)
print("  DATA CLEANING — PART 1B")
print("=" * 55)

df_clean = df.copy()
steps = []

# ── Step 1: Remove lead_key duplicates ──
before = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['lead_key'], keep='first')
removed = before - len(df_clean)
steps.append(('Remove lead_key duplicates', before, len(df_clean), removed))
print(f"✅ Step 1 — Dedup on lead_key: removed {removed:,} rows")

# ── Step 2: Standardise text fields ──
df_clean['business_name'] = df_clean['business_name'].str.strip().str.title()
df_clean['suburb'] = df_clean['suburb'].str.strip().str.lower()
df_clean['state'] = df_clean['state'].str.strip().str.upper()
df_clean['sector_level_1'] = df_clean['sector_level_1'].str.strip()
df_clean['sector_level_2'] = df_clean['sector_level_2'].str.strip()
df_clean['sector_level_3'] = df_clean['sector_level_3'].str.strip()
print(f"✅ Step 2 — Standardised text casing across all fields")

# ── Step 3: Phone analysis with phonenumbers library ──

# 3a. Extract phone from lead_key
df_clean['phone_from_leadkey'] = df_clean['lead_key'].str.replace('L_', '', n=1)

# 3b. Parse & validate with phonenumbers
def classify_phone(raw_phone):
    """Parse a raw phone string, return (parsed_number, is_valid, is_possible, phone_type)."""
    if pd.isna(raw_phone):
        return None, False, False, None
    stripped = re.sub(r'[\s()\-+]', '', str(raw_phone))
    if not stripped:
        return None, False, False, None

    for attempt in [str(raw_phone), '+' + stripped, stripped]:
        try:
            parsed = phonenumbers.parse(attempt, 'AU')
            valid = phonenumbers.is_valid_number(parsed)
            possible = phonenumbers.is_possible_number(parsed)
            ptype = phonenumbers.number_type(parsed)
            return parsed, valid, possible, ptype
        except phonenumbers.NumberParseException:
            continue
    return None, False, False, None

# Apply to phone column
phone_results = df_clean['phone'].apply(classify_phone)
df_clean['_phone_parsed']   = phone_results.apply(lambda x: x[0])
df_clean['_phone_valid']    = phone_results.apply(lambda x: x[1])
df_clean['_phone_possible'] = phone_results.apply(lambda x: x[2])
df_clean['_phone_type_raw'] = phone_results.apply(lambda x: x[3])

# Apply to lead_key phone
lk_results = df_clean['phone_from_leadkey'].apply(classify_phone)
df_clean['_lk_parsed']   = lk_results.apply(lambda x: x[0])
df_clean['_lk_valid']    = lk_results.apply(lambda x: x[1])
df_clean['_lk_possible'] = lk_results.apply(lambda x: x[2])
df_clean['_lk_type_raw'] = lk_results.apply(lambda x: x[3])

# 3c. Map phone types to BD-friendly labels
# Note: AU 13xx short numbers are classified as SHARED_COST by phonenumbers
TYPE_MAP = {
    PhoneNumberType.MOBILE: 'mobile',
    PhoneNumberType.FIXED_LINE: 'landline',
    PhoneNumberType.FIXED_LINE_OR_MOBILE: 'landline',
    PhoneNumberType.TOLL_FREE: 'toll_free',
    PhoneNumberType.SHARED_COST: 'shared_cost',
    PhoneNumberType.PREMIUM_RATE: 'premium',
    PhoneNumberType.UAN: 'shared_cost',
}

def map_phone_type(ptype):
    if ptype is None:
        return 'unknown'
    return TYPE_MAP.get(ptype, 'unknown')

# Use lead_key type as primary (it has the real number), fall back to phone column
df_clean['phone_type'] = df_clean['_lk_type_raw'].apply(map_phone_type)
# Where lead_key type is unknown, try phone column
mask_unknown_lk = df_clean['phone_type'] == 'unknown'
df_clean.loc[mask_unknown_lk, 'phone_type'] = df_clean.loc[mask_unknown_lk, '_phone_type_raw'].apply(map_phone_type)

# 3d. Cross-reference lead_key vs phone column — match_score
def compute_match_score(row):
    phone_raw = re.sub(r'[\s()\-+]', '', str(row['phone'])) if pd.notna(row['phone']) else ''
    lk_raw = str(row['phone_from_leadkey']) if pd.notna(row['phone_from_leadkey']) else ''
    if not phone_raw or not lk_raw:
        return 0.0
    # Both are placeholder patterns
    placeholder_patterns = ['611300000000', '611800000000', '000000000000']
    if phone_raw in placeholder_patterns and lk_raw in placeholder_patterns:
        return 0.0
    if phone_raw == lk_raw:
        return 1.0
    # Check prefix match (masked trailing digits)
    min_len = min(len(phone_raw), len(lk_raw))
    if min_len >= 6 and phone_raw[:6] == lk_raw[:6]:
        # Trailing zeros in phone suggest masking
        if phone_raw.endswith('000000') or phone_raw.endswith('0000'):
            return 0.5
        return 0.5  # prefix match but different trailing
    return 0.2

df_clean['phone_match_score'] = df_clean.apply(compute_match_score, axis=1)

# 3e. Frequency analysis
phone_freq = df_clean['phone'].value_counts()
df_clean['phone_frequency'] = df_clean['phone'].map(phone_freq).fillna(0).astype(int)

# 3f. Pattern checks
def has_bad_pattern(phone_str):
    """Flag numbers with 6+ trailing zeros or all repeating digits."""
    if pd.isna(phone_str):
        return True
    s = re.sub(r'[\s()\-+]', '', str(phone_str))
    if not s:
        return True
    if re.search(r'0{6,}$', s):
        return True
    if len(set(s)) == 1 and len(s) > 3:
        return True
    return False

df_clean['_phone_bad_pattern'] = df_clean['phone'].apply(has_bad_pattern)
df_clean['_lk_bad_pattern'] = df_clean['phone_from_leadkey'].apply(has_bad_pattern)

# 3g. Assign phone_quality (Level 1 / Level 2 / Level 3)
def assign_phone_quality(row):
    is_valid = row['_phone_valid'] or row['_lk_valid']
    is_possible = row['_phone_possible'] or row['_lk_possible']
    match_score = row['phone_match_score']
    freq = row['phone_frequency']
    bad_pattern = row['_phone_bad_pattern'] and row['_lk_bad_pattern']

    # Level 1 (fake) conditions
    if bad_pattern:
        return 'Level 1'
    if match_score == 0.0:
        return 'Level 1'
    if freq > 10:
        return 'Level 1'
    if not is_valid and not is_possible:
        return 'Level 1'

    # Level 3 (valid) conditions
    if is_valid and match_score == 1.0 and freq <= 2 and not row['_phone_bad_pattern']:
        return 'Level 3'

    # Level 2 (suspicious) — everything else
    return 'Level 2'

df_clean['phone_quality'] = df_clean.apply(assign_phone_quality, axis=1)

# 3h. Assign reachability_score (0-30)
def assign_reachability(row):
    quality = row['phone_quality']
    ptype = row['phone_type']

    if quality == 'Level 1':
        return 0
    if quality == 'Level 2':
        return 3
    # Level 3
    score_map = {
        'mobile': 30,
        'landline': 25,
        'shared_cost': 20,
        'toll_free': 10,
        'premium': 0,
        'unknown': 15,
    }
    return score_map.get(ptype, 15)

df_clean['reachability_score'] = df_clean.apply(assign_reachability, axis=1)

# Drop internal helper columns
internal_cols = [c for c in df_clean.columns if c.startswith('_')]
df_clean.drop(columns=internal_cols, inplace=True)

# Summary
print(f"✅ Step 3 — Phone analysis (phonenumbers library):")
print(f"   phone_quality distribution:")
for level, count in df_clean['phone_quality'].value_counts().sort_index().items():
    print(f"     {level}: {count:>8,} ({count/len(df_clean)*100:.1f}%)")
print(f"   phone_type distribution:")
for ptype, count in df_clean['phone_type'].value_counts().items():
    print(f"     {ptype:>12s}: {count:>8,}")
print(f"   reachability_score: mean={df_clean['reachability_score'].mean():.1f}, median={df_clean['reachability_score'].median():.0f}")

# ── Step 4: Non-target filter ──
non_target_keywords = ['Retirement', 'Equestrian', 'NGO', 'Construction',
                       'Community Health', 'Religious', 'Fabrication',
                       'Marriage Celebrant', 'Recycling']
df_clean['kpay_relevant'] = ~df_clean['sector_level_3'].str.contains(
    '|'.join(non_target_keywords), case=False, na=False)
non_target_removed = (~df_clean['kpay_relevant']).sum()
print(f"✅ Step 4 — Non-target filter: flagged {non_target_removed:,} irrelevant merchants")

# ── Step 5: Contactability / data completeness helpers ──
df_clean['has_address'] = df_clean['address'].notna()
df_clean['has_sector'] = df_clean['sector_level_1'].notna()
print(f"✅ Step 5 — Data completeness flags created")

# ── Step 6: Flag name+suburb dupes ──
df_clean['name_suburb_dup'] = df_clean.duplicated(
    subset=['business_name', 'suburb'], keep='first')
dup_flagged = df_clean['name_suburb_dup'].sum()
print(f"✅ Step 6 — Flagged {dup_flagged:,} name+suburb duplicates (kept, not deleted)")

  DATA CLEANING — PART 1B
✅ Step 1 — Dedup on lead_key: removed 0 rows


✅ Step 2 — Standardised text casing across all fields


✅ Step 3 — Phone analysis (phonenumbers library):
   phone_quality distribution:
     Level 1:      406 (0.2%)
     Level 2:  111,935 (56.0%)
     Level 3:   87,658 (43.8%)
   phone_type distribution:
         landline:  147,820
           mobile:   46,536
          unknown:    2,724
      shared_cost:    2,209
        toll_free:      710
   reachability_score: mean=13.4, median=3


✅ Step 4 — Non-target filter: flagged 2,394 irrelevant merchants
✅ Step 5 — Data completeness flags created
✅ Step 6 — Flagged 8,040 name+suburb duplicates (kept, not deleted)


## 5. Cleaning Funnel

In [9]:
# Working on relevant records only
df_relevant = df_clean[df_clean['kpay_relevant']].copy()

print("\n📊 CLEANING FUNNEL:")
print(f"  Raw records:                      {len(df):>8,}")
print(f"  After dedup (lead_key):           {len(df_clean):>8,}")
print(f"  After non-target filter:          {len(df_relevant):>8,}")

# Phone quality breakdown
l3 = (df_relevant['phone_quality'] == 'Level 3').sum()
l2 = (df_relevant['phone_quality'] == 'Level 2').sum()
l1 = (df_relevant['phone_quality'] == 'Level 1').sum()
print(f"\n  Phone quality:")
print(f"    Level 3 (valid):                {l3:>8,} ({l3/len(df_relevant)*100:.1f}%)")
print(f"    Level 2 (suspicious):           {l2:>8,} ({l2/len(df_relevant)*100:.1f}%)")
print(f"    Level 1 (fake):                 {l1:>8,} ({l1/len(df_relevant)*100:.1f}%)")

# Reachability breakdown
contactable = (df_relevant['reachability_score'] > 0).sum()
print(f"\n  Contactable (reachability > 0):   {contactable:>8,} ({contactable/len(df_relevant)*100:.1f}%)")
print(f"  With address:                     {df_relevant['has_address'].sum():>8,}")

actionable = l3 + l2  # Level 2+ are worth attempting
actionable_pct = actionable / len(df) * 100
print(f"""
💡 ACTIONABLE LEAD RATE AFTER CLEANING: {actionable_pct:.1f}%
   These are Level 2+ phone quality in KPay-relevant sectors.
   Level 3 (high confidence): {l3:,} | Level 2 (worth trying): {l2:,}
""")


📊 CLEANING FUNNEL:
  Raw records:                       199,999
  After dedup (lead_key):            199,999
  After non-target filter:           197,605

  Phone quality:
    Level 3 (valid):                  87,148 (44.1%)
    Level 2 (suspicious):            110,054 (55.7%)
    Level 1 (fake):                      403 (0.2%)

  Contactable (reachability > 0):    197,202 (99.8%)
  With address:                      197,553

💡 ACTIONABLE LEAD RATE AFTER CLEANING: 98.6%
   These are Level 2+ phone quality in KPay-relevant sectors.
   Level 3 (high confidence): 87,148 | Level 2 (worth trying): 110,054



##  6. KMF Scoring

In [10]:
print("=" * 55)
print("  KMF SCORE — PART 2A (REBALANCED)")
print("=" * 55)

# --- REACHABILITY (40 pts) ---
# reachability_score (0-30) from phone analysis + has_address (0 or 10)
df_relevant['reach_address'] = df_relevant['has_address'].astype(int) * 10
df_relevant['reachability']  = df_relevant['reachability_score'] + df_relevant['reach_address']

# --- SECTOR PRIORITY (35 pts) ---
sector_map = {
    'F&B': 35,
    'Beauty & Wellness': 26,
    'Retail': 26,
    'Professional Services': 12,
    'Others': 9
}
df_relevant['sector_score'] = df_relevant['sector_level_1'].map(sector_map).fillna(6)

# --- GEO EFFICIENCY (15 pts) ---
tier_a = ['hurstville','chatswood','burwood','parramatta','cabramatta',
          'eastwood','ashfield','strathfield','campsie','haymarket']
tier_b = ['bankstown','kogarah','roselands','rhodes','merrylands',
          'auburn','lidcombe','granville']

def geo_score(suburb):
    if pd.isna(suburb):
        return 1
    s = str(suburb).lower().strip()
    if s in tier_a:
        return 15
    elif s in tier_b:
        return 10
    elif 'sydney' in s or 'nsw' in s:
        return 5
    else:
        return 1

df_relevant['geo_score'] = df_relevant['suburb'].apply(geo_score)

# --- DATA COMPLETENESS (10 pts) ---
# has_address (3) + has_sector (3) + phone_match_score contribution (4)
df_relevant['data_completeness'] = (
    df_relevant['has_address'].astype(int) * 3 +
    df_relevant['has_sector'].astype(int) * 3 +
    (df_relevant['phone_match_score'] * 4).round().astype(int)
)

# --- FINAL KMF SCORE (100 pts) ---
df_relevant['kmf_score'] = (
    df_relevant['reachability'] +
    df_relevant['sector_score'] +
    df_relevant['geo_score'] +
    df_relevant['data_completeness']
)

# --- TIER ASSIGNMENT ---
def assign_tier(score):
    if score >= 70:
        return 'Tier 1'
    elif score >= 40:
        return 'Tier 2'
    else:
        return 'Tier 3'

df_relevant['tier'] = df_relevant['kmf_score'].apply(assign_tier)

# Summary
print("\n📊 KMF BUDGET: Reachability(40) + Sector(35) + Geo(15) + Completeness(10) = 100")
print(f"\n📊 KMF SCORE DISTRIBUTION:")
print(f"   Mean:   {df_relevant['kmf_score'].mean():.1f}")
print(f"   Median: {df_relevant['kmf_score'].median():.0f}")
print(f"   Std:    {df_relevant['kmf_score'].std():.1f}")
print(f"   Min:    {df_relevant['kmf_score'].min():.0f}")
print(f"   Max:    {df_relevant['kmf_score'].max():.0f}")

print("\n📊 TIER BREAKDOWN:")
tier_summary = df_relevant.groupby('tier').agg(
    Count=('kmf_score', 'count'),
    Avg_KMF=('kmf_score', 'mean'),
    Min_KMF=('kmf_score', 'min'),
    Max_KMF=('kmf_score', 'max')
).round(1)
print(tier_summary)

print("\n📊 COMPONENT DISTRIBUTIONS:")
for comp in ['reachability', 'sector_score', 'geo_score', 'data_completeness']:
    s = df_relevant[comp]
    print(f"   {comp:>20s}: mean={s.mean():.1f}, min={s.min():.0f}, max={s.max():.0f}")

print("\n📊 TIER 1 — TOP SECTORS:")
print(df_relevant[df_relevant['tier']=='Tier 1']['sector_level_1'].value_counts().head(8))

print("\n📊 TIER 1 — TOP SUBURBS:")
print(df_relevant[df_relevant['tier']=='Tier 1']['suburb'].value_counts().head(10))

  KMF SCORE — PART 2A (REBALANCED)



📊 KMF BUDGET: Reachability(40) + Sector(35) + Geo(15) + Completeness(10) = 100

📊 KMF SCORE DISTRIBUTION:
   Mean:   47.1
   Median: 47
   Std:    17.3
   Min:    11
   Max:    100

📊 TIER BREAKDOWN:
        Count  Avg_KMF  Min_KMF  Max_KMF
tier                                    
Tier 1  29283     78.7     70.0    100.0
Tier 2  83063     52.9     40.0     69.0
Tier 3  85259     30.5     11.0     39.0

📊 COMPONENT DISTRIBUTIONS:
           reachability: mean=23.5, min=3, max=40
           sector_score: mean=14.0, min=6, max=35
              geo_score: mean=1.8, min=1, max=15
      data_completeness: mean=7.8, min=1, max=10

📊 TIER 1 — TOP SECTORS:


sector_level_1
Retail                   12922
F&B                      10081
Beauty & Wellness         5871
Professional Services      263
Others                     108
Restaurant                   5
Health & Beauty              4
Retail Shopping              3
Name: count, dtype: int64

📊 TIER 1 — TOP SUBURBS:
suburb
sydney         2533
parramatta      692
chatswood       535
surry hills     466
liverpool       357
bankstown       335
castle hill     327
blacktown       314
burwood         292
brookvale       279
Name: count, dtype: int64


##  7.  Export Cleaned File

In [11]:
# Export
output_path = "data/kpay_cleaned.csv"
df_relevant.to_csv(output_path, index=False)

print(f"✅ Exported: {output_path}")
print(f"   Rows: {len(df_relevant):,}")
print(f"   Columns: {len(df_relevant.columns)}")
print(f"\n   New columns added:")
new_cols = ['phone_from_leadkey', 'phone_type', 'phone_quality',
            'phone_match_score', 'phone_frequency', 'reachability_score',
            'kpay_relevant', 'has_address', 'has_sector', 'name_suburb_dup',
            'reachability', 'reach_address', 'sector_score', 'geo_score',
            'data_completeness', 'kmf_score', 'tier']
for c in new_cols:
    print(f"   → {c}")

print("\n🎉 Part 1A + 1B + KMF Scoring (rebalanced) complete!")
print("   Open kpay_cleaned.csv and check a few rows before moving to slides.")

✅ Exported: data/kpay_cleaned.csv
   Rows: 197,605
   Columns: 28

   New columns added:
   → phone_from_leadkey
   → phone_type
   → phone_quality
   → phone_match_score
   → phone_frequency
   → reachability_score
   → kpay_relevant
   → has_address
   → has_sector
   → name_suburb_dup
   → reachability
   → reach_address
   → sector_score
   → geo_score
   → data_completeness
   → kmf_score
   → tier

🎉 Part 1A + 1B + KMF Scoring (rebalanced) complete!
   Open kpay_cleaned.csv and check a few rows before moving to slides.
